### Fields
- AdjEM = AdjO - AdjD
- Luck
- SoS (conference) AdjEM = OppO - OppD
- NCSOS (non-conference)

### Goal:
My goal is to take just the fields presented in the **Fields** heading and weigh each appropriately to get a score. The higher of the two scores between teams will be the one that advances. This heuristic should mirror the outcomes of previous tournaments.

I believe I can take two teams input fields and set a classifier by whether team 1 beats team 2.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('kenpom.csv')

# Reduced dataset
columns = ['Year', 'Team', 'Pyth', 'Luck', 'SOS Pyth', 'NCSOS Pyth']
small_df = df[columns]

# Cleanup Strings
small_df['Team'] = small_df['Team'].str.strip()

small_df.head()

## Combine Historical Data (2012-2018)

In [ ]:
df1 = pd.read_csv('tournament_results/2002_2012.csv')
df2 = pd.read_csv('tournament_results/2013_2018.csv')
historical_data = pd.concat([df1, df2],
                            ignore_index=True)
historical_data.shape

I have to cleanup team names to get appropriate matching to pomeroy ratings

In [ ]:
all_historical_names = set(pd.concat([historical_data['Team1'],historical_data['Team2']]))
all_pomeroy_names = set(small_df['Team'])

In [ ]:
# Number of teams not found in historical teams
len(all_pomeroy_names - all_historical_names)

In [ ]:
missing = []

# Find messed up names in historical data
for element in all_historical_names:
    if element in all_pomeroy_names:
        continue
    else:
        missing.append(element)

missing = set(missing)
print(len(missing))
print(missing)

In [81]:
corrections = pd.read_csv('ncaa_name_conversion.csv')
corrections.head()

,historical,pomeroy
0,Saint Mary's Calif.,Saint Mary's
1,St. Peter's,Saint Peter's
2,Jackson St,Jackson St.
3,Texas-Arlington,UT Arlington
4,ETSU,East Tennessee St.


In [ ]:
corrections.loc[corrections['historical'] == "St. Peter's", 'pomeroy'].values

In [ ]:
# Reformat team names in historical data to match formatting of Pomeroy ratings
for i in range(len(historical_data)):
    team1 = historical_data.loc[i, 'Team1']
    team2 = historical_data.loc[i, 'Team2']
    if team1 in missing:
        historical_data.loc[i, 'Team1'] = corrections.loc[corrections['historical'] == team1, 'pomeroy'].values
    if team2 in missing:
        historical_data.loc[i, 'Team2'] = corrections.loc[corrections['historical'] == team2, 'pomeroy'].values

### Create classifier dataframes

In [ ]:
# VERSION 2.0
classification_df = pd.DataFrame(columns=['Team1_Pyth', 'Team1_Luck','Team1_SOS_Pyth', 'Team1_NCSOS_Pyth',
                                          'Team2_Pyth', 'Team2_Luck','Team2_SOS_Pyth', 'Team2_NCSOS_Pyth',
                                          'outcome'])

for i in range(len(historical_data)):

    # Isolate matchup data
    team1 = historical_data['Team1'][i]
    team2 = historical_data['Team2'][i]
    year = historical_data['Year'][i]

    columns = ['Pyth', 'Luck', 'SOS Pyth', 'NCSOS Pyth']
    team1_data = small_df[(small_df['Team'] == team1) &
                          (small_df['Year'] == year)][columns].reset_index(drop=True)
    team2_data = small_df[(small_df['Team'] == team2) &
                          (small_df['Year'] == year)][columns].reset_index(drop=True)

    # Rename columns
    team1_data = team1_data.rename(columns={'Pyth': 'Team1_Pyth',
                                       'Luck': 'Team1_Luck',
                                       'SOS Pyth': 'Team1_SOS_Pyth',
                                       'NCSOS Pyth': 'Team1_NCSOS_Pyth'})
    team2_data = team2_data.rename(columns={'Pyth': 'Team2_Pyth',
                                       'Luck': 'Team2_Luck',
                                       'SOS Pyth': 'Team2_SOS_Pyth',
                                       'NCSOS Pyth': 'Team2_NCSOS_Pyth'})

    # Merge Data
    feature_vector = pd.concat([team1_data, team2_data],
                               axis=1)

    # Label if Team1 won (1) or not (0)
    feature_vector['outcome'] = 1
        
    # Append to main dataframe
    classification_df = pd.concat([classification_df, feature_vector],
                                   ignore_index=True)
    
    #
    # Make counterset of data
    #
    
    # Isolate matchup data
    team1 = historical_data['Team2'][i]  # Switching data
    team2 = historical_data['Team1'][i]
    year = historical_data['Year'][i]


    columns = ['Pyth', 'Luck', 'SOS Pyth', 'NCSOS Pyth']
    team1_data = small_df[(small_df['Team'] == team1) &
                          (small_df['Year'] == year)][columns].reset_index(drop=True)
    team2_data = small_df[(small_df['Team'] == team2) &
                          (small_df['Year'] == year)][columns].reset_index(drop=True)

    # Rename columns
    team1_data = team1_data.rename(columns={'Pyth': 'Team1_Pyth',
                                       'Luck': 'Team1_Luck',
                                       'SOS Pyth': 'Team1_SOS_Pyth',
                                       'NCSOS Pyth': 'Team1_NCSOS_Pyth'})
    team2_data = team2_data.rename(columns={'Pyth': 'Team2_Pyth',
                                       'Luck': 'Team2_Luck',
                                       'SOS Pyth': 'Team2_SOS_Pyth',
                                       'NCSOS Pyth': 'Team2_NCSOS_Pyth'})

    # Merge Data
    feature_vector = pd.concat([team1_data, team2_data],
                               axis=1)

    # Label if Team1 won (1) or not (0)
    feature_vector['outcome'] = 0
        
    # Append to main dataframe
    classification_df = pd.concat([classification_df, feature_vector],
                                   ignore_index=True)

In [ ]:
# Gonna drop messed up rows for now
clean = classification_df[~pd.isnull(classification_df).any(axis=1)]
class_reduced_df = classification_df.iloc[clean.index,:]

# Clean up data type
class_reduced_df['outcome'] = class_reduced_df['outcome'].astype('int')

# Reset Index
class_reduced_df.reset_index(inplace=True, drop=True)

print(class_reduced_df.shape)

class_reduced_df.to_csv('game_features.csv', index=False)

# Model Development

In [200]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'    # To precent dead kernel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import KFold

In [201]:
# Load Feature data
df = pd.read_csv('kenpom.csv')

# Reduced dataset
columns = ['Year', 'Team', 'Pyth', 'Luck', 'SOS Pyth', 'NCSOS Pyth']
pomeroy_df = df[columns]

# Cleanup Strings
pomeroy_df['Team'] = pomeroy_df['Team'].str.strip()

# Load training data
game_df = pd.read_csv('game_features.csv')
data_x = game_df.iloc[:, 0:8]
data_y = game_df.iloc[:, 8]

# # Subset data for training
# train_x, test_x, train_y, test_y = train_test_split(data_x, data_y,
#                                                     test_size=0.2)

/Users/jad/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [202]:
def log_loss(probabilities, y):
    num_samples = len(probabilities)
    nat_log_prob = np.log(probabilities)
    nat_log_minus_prob = np.log(1 - probabilities)
    factor_1 = y
    factor_2 = 1 - y
    
    log_loss = np.sum(-(1 / num_samples) * ((factor_1 * nat_log_prob) + (factor_2 * nat_log_minus_prob)))
    return log_loss

In [203]:
def cross_validation(model):
    # Subset data
    X = game_df.iloc[:, 0:8].values
    y = game_df.iloc[:, 8].values

    # K-fold CV
    kf = KFold(n_splits=5, random_state=42, shuffle=True)

    scores = []
    log_loss_scores = []
    for train_index, test_index, in kf.split(data_x):
        # Train
        train_x, train_y = X[train_index], y[train_index]
        clf = model
        clf.fit(train_x, train_y)

        # Test
        test_x, test_y = X[test_index], y[test_index]
        score = clf.score(test_x, test_y)
        scores.append(score)
        
        # Get probabilites for test predictions
        prob_matrix = clf.predict_proba(test_x)
        
        # Get probability for class = 1 (columne 2)
        probabilities = prob_matrix.flatten('F')[prob_matrix.shape[0]:]
        
        log_loss_score = log_loss(probabilities, test_y)
        log_loss_scores.append(log_loss_score)
    
    print("Mean Accuracy:", np.mean(scores))
    print("Mean Log Loss:", np.mean(log_loss_scores))
    print("--------------")

In [204]:
# Models
clf1 = LogisticRegression(random_state=0, solver='lbfgs',
                          multi_class='multinomial')

clf2 = SVC(kernel='rbf',
           C=0.5,
           gamma='auto', probability=True)

clf3 = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)

clf4 = XGBClassifier(max_depth=3)

In [205]:
# Run CV
cross_validation(clf1)
cross_validation(clf2)
cross_validation(clf3)
cross_validation(clf4)

Mean Accuracy: 0.7520974553901383
Mean Log Loss: 0.5001320622893696
--------------
Mean Accuracy: 0.641969696969697
Mean Log Loss: 0.6428754373264529
--------------
Mean Accuracy: 0.6949271460247071
Mean Log Loss: 0.6067909756746096
--------------
Mean Accuracy: 0.7235096610706366
Mean Log Loss: 0.5194889126165709
--------------


In [206]:
# XGBOOST data
X = game_df.iloc[:, 0:8].values
y = game_df.iloc[:, 8].values

# Train with all data
clf1.fit(data_x, data_y)
clf2.fit(data_x, data_y)
clf3.fit(data_x, data_y)
clf4.fit(data_x, data_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [34]:
def predict_game(team1, team2, clf):
    year = 2019

    columns = ['Pyth', 'Luck', 'SOS Pyth', 'NCSOS Pyth']
    team1_data = pomeroy_df[(pomeroy_df['Team'] == team1) &
                          (pomeroy_df['Year'] == year)][columns].reset_index(drop=True)
    team2_data = pomeroy_df[(pomeroy_df['Team'] == team2) &
                          (pomeroy_df['Year'] == year)][columns].reset_index(drop=True)

    # Rename columns
    team1_data = team1_data.rename(columns={'Pyth': 'Team1_Pyth',
                                       'Luck': 'Team1_Luck',
                                       'SOS Pyth': 'Team1_SOS_Pyth',
                                       'NCSOS Pyth': 'Team1_NCSOS_Pyth'})
    team2_data = team2_data.rename(columns={'Pyth': 'Team2_Pyth',
                                       'Luck': 'Team2_Luck',
                                       'SOS Pyth': 'Team2_SOS_Pyth',
                                       'NCSOS Pyth': 'Team2_NCSOS_Pyth'})

    # Merge Data
    single_game = pd.concat([team1_data, team2_data],
                               axis=1)

    team1_prob_1 = clf.predict_proba(single_game)[0,0]
    team2_prob_1 = clf.predict_proba(single_game)[0,1]
    
    
    # Change teams
    team1_data = pomeroy_df[(pomeroy_df['Team'] == team2) &
                          (pomeroy_df['Year'] == year)][columns].reset_index(drop=True)
    team2_data = pomeroy_df[(pomeroy_df['Team'] == team1) &
                          (pomeroy_df['Year'] == year)][columns].reset_index(drop=True)

    # Rename columns
    team1_data = team1_data.rename(columns={'Pyth': 'Team1_Pyth',
                                       'Luck': 'Team1_Luck',
                                       'SOS Pyth': 'Team1_SOS_Pyth',
                                       'NCSOS Pyth': 'Team1_NCSOS_Pyth'})
    team2_data = team2_data.rename(columns={'Pyth': 'Team2_Pyth',
                                       'Luck': 'Team2_Luck',
                                       'SOS Pyth': 'Team2_SOS_Pyth',
                                       'NCSOS Pyth': 'Team2_NCSOS_Pyth'})

    # Merge Data
    single_game = pd.concat([team1_data, team2_data],
                               axis=1)
    
    team1_prob_2 = clf.predict_proba(single_game)[0,1]
    team2_prob_2 = clf.predict_proba(single_game)[0,0]
    
    team1_prob_true = (team1_prob_1 + team1_prob_2) / 2
    team2_prob_true = (team2_prob_1 + team2_prob_2) / 2
    
    print(team1, "probability:", team1_prob_true)
    print(team2, "probability:", team2_prob_true)
    print('----')

In [37]:
# 11 Seed Game
predict_game('Temple', 'Belmont', clf1)
predict_game('Temple', 'Belmont', clf4)

# 16 Seed Game
predict_game('Fairleigh Dickinson', 'Prairie View A&M', clf1)
predict_game('Fairleigh Dickinson', 'Prairie View A&M', clf4)

Temple probability: 0.5921767511540075
Belmont probability: 0.4078232488459925
----
Temple probability: 0.6302804350852966
Belmont probability: 0.36971956491470337
----
Fairleigh Dickinson probability: 0.5928062021482667
Prairie View A&M probability: 0.40719379785173326
----
Fairleigh Dickinson probability: 0.5203211307525635
Prairie View A&M probability: 0.47967883944511414
----


In [38]:
# 16 Seed Game
predict_game('North Carolina Central', 'North Dakota St.', clf1)
predict_game('North Carolina Central', 'North Dakota St.', clf4)

# 16 Seed Game
predict_game("St. John's", 'Arizona St.', clf1)
predict_game("St. John's", 'Arizona St.', clf4)

North Carolina Central probability: 0.8663655490719193
North Dakota St. probability: 0.13363445092808068
----
North Carolina Central probability: 0.8346788883209229
North Dakota St. probability: 0.16532108187675476
----
St. John's probability: 0.5775836417916886
Arizona St. probability: 0.4224163582083113
----
St. John's probability: 0.6191689968109131
Arizona St. probability: 0.3808310031890869
----


In [ ]:
# Find team in pomeroy data
small_df[small_df['Team'] == "St. John's"]

In [50]:
team_spellings = pd.read_csv('TeamSpellings.csv')
print(team_spellings.shape)
team_spellings.head()

(1141, 2)


,TeamNameSpelling,TeamID
0,a&m-corpus chris,1394
1,a&m-corpus christi,1394
2,abilene chr,1101
3,abilene christian,1101
4,abilene-christian,1101


### Prep submission samples for predictions

In [229]:
# Sample submission
sample_submission = pd.read_csv('SampleSubmissionStage2.csv')
sub = sample_submission.copy()

# Extract team ID's
sub['Year'], sub['Team1ID'], sub['Team2ID'] = sample_submission['ID'].str.split('_').str
sub[['Year', 'Team1ID', 'Team2ID']] = sub[['Year', 'Team1ID', 'Team2ID']].apply(pd.to_numeric)
print(sub.shape)
sub.head()

(2278, 5)


,ID,Pred,Year,Team1ID,Team2ID
0,2019_1101_1113,0.5,2019,1101,1113
1,2019_1101_1120,0.5,2019,1101,1120
2,2019_1101_1124,0.5,2019,1101,1124
3,2019_1101_1125,0.5,2019,1101,1125
4,2019_1101_1133,0.5,2019,1101,1133


In [230]:
# Get all teamID's in tournament
all_ids = set()

for element1, element2 in zip(sub['Team1ID'], sub['Team2ID']):
    all_ids.add(element1)
    all_ids.add(element2)

In [231]:
# Teams
team_codes = pd.read_csv('Teams.csv')
team_codes.rename(columns={"TeamName": 'Team'}, inplace=True)
team_codes['Year'] = 2019

# Subset for only teams in the tournament
team_codes = team_codes[team_codes['TeamID'].isin(all_ids)]

# Combine with pomeroy features
team_codes_pomeroy = pd.merge(team_codes, pomeroy_df,
                              how='left',
                              on=['Team', 'Year'])
team_codes_pomeroy.shape
team_codes_pomeroy.head()

,TeamID,Team,FirstD1Season,LastD1Season,Year,Pyth,Luck,SOS Pyth,NCSOS Pyth
0,1101,Abilene Chr,2014,2019,2019,NaN,NaN,NaN,NaN
1,1113,Arizona St,1985,2019,2019,NaN,NaN,NaN,NaN
2,1120,Auburn,1985,2019,2019,23.33,0.003,10.89,2.82
3,1124,Baylor,1985,2019,2019,15.29,-0.032,9.18,-4.20
4,1125,Belmont,2000,2019,2019,13.08,0.041,-3.22,3.46


In [232]:
# Team spelling changes
corrections = pd.read_csv('ncaa_name_conversion.csv')

# Reformat team names in historical data to match formatting of Pomeroy ratings
for i in range(len(team_codes_pomeroy)):
    teamluck = team_codes_pomeroy.loc[i, 'Luck']
    team = team_codes_pomeroy.loc[i, 'Team']
    if np.isnan(teamluck):
#         print(team)
#         print(corrections.loc[corrections['historical'] == team, 'pomeroy'].values)
        team_codes_pomeroy.loc[i, 'Team'] = corrections.loc[corrections['historical'] == team, 'pomeroy'].values

# Drop Columns for proper merge
team_codes_pomeroy = team_codes_pomeroy[['TeamID', 'Team', 'Year']]
    
# Redo Merge to get pomeroy features
team_codes_pomeroy = pd.merge(team_codes_pomeroy, pomeroy_df,
                              how='left',
                              on=['Team', 'Year'])

team_codes_pomeroy.to_csv('team_features.csv', index=False)

In [233]:
sub.head()

,ID,Pred,Year,Team1ID,Team2ID
0,2019_1101_1113,0.5,2019,1101,1113
1,2019_1101_1120,0.5,2019,1101,1120
2,2019_1101_1124,0.5,2019,1101,1124
3,2019_1101_1125,0.5,2019,1101,1125
4,2019_1101_1133,0.5,2019,1101,1133


In [236]:
team_codes_pomeroy.head()

,TeamID,Team,Year,Pyth,Luck,SOS Pyth,NCSOS Pyth
0,1101,Abilene Christian,2019,2.09,0.087,-9.03,-4.51
1,1113,Arizona St.,2019,11.21,0.046,4.81,1.80
2,1120,Auburn,2019,23.33,0.003,10.89,2.82
3,1124,Baylor,2019,15.29,-0.032,9.18,-4.20
4,1125,Belmont,2019,13.08,0.041,-3.22,3.46


#### CREATE FEATURE VECTORS FOR PREDICTIONS

In [237]:
all_games = pd.DataFrame(columns=['Team1_Pyth', 'Team1_Luck','Team1_SOS_Pyth', 'Team1_NCSOS_Pyth',
                                  'Team2_Pyth', 'Team2_Luck','Team2_SOS_Pyth', 'Team2_NCSOS_Pyth'])

for i in range(len(sub)):
    print(i)
    # Create features for each game
    team1 = sub.loc[i, 'Team1ID']
    team2 = sub.loc[i, 'Team2ID']

    team1_data = team_codes_pomeroy[team_codes_pomeroy['TeamID'] == team1]
    team2_data = team_codes_pomeroy[team_codes_pomeroy['TeamID'] == team2]
    
    # Drop columns (required for model input)
    team1_data.drop(columns=['TeamID', 'Team', 'Year'], inplace=True)
    team2_data.drop(columns=['TeamID', 'Team', 'Year'], inplace=True)
    
    # Reset index
    team1_data.reset_index(inplace=True, drop=True)
    team2_data.reset_index(inplace=True, drop=True)

    # Rename columns (required for model input)
    team1_data = team1_data.rename(columns={'Pyth': 'Team1_Pyth',
                                       'Luck': 'Team1_Luck',
                                       'SOS Pyth': 'Team1_SOS_Pyth',
                                       'NCSOS Pyth': 'Team1_NCSOS_Pyth'})
    team2_data = team2_data.rename(columns={'Pyth': 'Team2_Pyth',
                                       'Luck': 'Team2_Luck',
                                       'SOS Pyth': 'Team2_SOS_Pyth',
                                       'NCSOS Pyth': 'Team2_NCSOS_Pyth'})

    # Merge Data
    single_game = pd.concat([team1_data, team2_data], axis=1)
    
    # Add to master dataframe
    all_games = pd.concat([all_games, single_game],
                                   ignore_index=True)

0
1
2
3


/Users/jad/miniconda3/envs/ml/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
27

1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063


In [238]:
# Check for any null values
all_games[all_games.isnull().values]

,Team1_Pyth,Team1_Luck,Team1_SOS_Pyth,Team1_NCSOS_Pyth,Team2_Pyth,Team2_Luck,Team2_SOS_Pyth,Team2_NCSOS_Pyth


In [239]:
all_games.shape

(2278, 8)

In [240]:
all_games.to_csv('all_game_features.csv', index=False)

In [265]:
# Get probability predictions
pred1 = clf1.predict_proba(all_games)
pred2 = clf4.predict_proba(all_games)

# Get probability for lower team ID winning
pred1_prob = pred1.flatten('F')[pred1.shape[0]:]
pred2_prob = pred2.flatten('F')[pred2.shape[0]:]

In [266]:
# Only 22/2278 where the class is different between the models
np.sum(clf1.predict(all_games) - clf4.predict(all_games))

22

In [267]:
# Create Submission files
submission1 = pd.read_csv('SampleSubmissionStage2.csv')
submission2 = pd.read_csv('SampleSubmissionStage2.csv')

# Add in predictions
submission1['Pred'] = pred1_prob
submission2['Pred'] = pred2_prob

# Write submissions
submission1.to_csv('submissions/LR.csv', index=False)
submission2.to_csv('submissions/XGBoost.csv', index=False)

In [268]:
pred1_prob

array([0.16524823, 0.02853722, 0.12495074, ..., 0.56698567, 0.91775682,
       0.89497895])

In [269]:
pred2_prob

array([0.10953595, 0.02156516, 0.2590544 , ..., 0.74770653, 0.933605  ,
       0.91083854], dtype=float32)

### Finding Games to Overide
- Duke: 1181 - NC Central: 1300 or - N Dakota St: 1295
- Virginia: 1438 - Gardner Webb: 1205
- Gonzaga: 1211 - F Dickinson: 1192
- North Carolina: 1314 - Iona: 1233

- Michigan St.: 1277 - Bradley: 1133
- Tennessee: 1397 - Colgate: 1159
- Michigan: 1276 - Montana: 1285
- Kentucky: 1246 - Abilene Chr: 1101

## OLD CODE

In [ ]:
classification_df = pd.DataFrame(columns=['Team1_Pyth', 'Team1_Luck','Team1_SOS_Pyth', 'Team1_NCSOS_Pyth',
                                          'Team2_Pyth', 'Team2_Luck','Team2_SOS_Pyth', 'Team2_NCSOS_Pyth',
                                          'outcome'])

for i in range(len(historical_data)):

    # Isolate matchup data
    team1 = historical_data['Team1'][i]
    team2 = historical_data['Team2'][i]
    winner = historical_data['Winner'][i]
    year = historical_data['Year'][i]
    
    columns = ['Pyth', 'Luck', 'SOS Pyth', 'NCSOS Pyth']
    team1_data = small_df[(small_df['Team'] == team1) &
                          (small_df['Year'] == year)][columns].reset_index(drop=True)
    team2_data = small_df[(small_df['Team'] == team2) &
                          (small_df['Year'] == year)][columns].reset_index(drop=True)

    # Rename columns
    team1_data = team1_data.rename(columns={'Pyth': 'Team1_Pyth',
                                       'Luck': 'Team1_Luck',
                                       'SOS Pyth': 'Team1_SOS_Pyth',
                                       'NCSOS Pyth': 'Team1_NCSOS_Pyth'})
    team2_data = team2_data.rename(columns={'Pyth': 'Team2_Pyth',
                                       'Luck': 'Team2_Luck',
                                       'SOS Pyth': 'Team2_SOS_Pyth',
                                       'NCSOS Pyth': 'Team2_NCSOS_Pyth'})

    # Merge Data
    feature_vector = pd.concat([team1_data, team2_data],
                               axis=1)

    # Label if Team1 won (1) or not (0)
    if winner == 'Team1':
        feature_vector['outcome'] = 1
    else:
        feature_vector['outcome'] = 0
        
    # Append to main dataframe
    classification_df = pd.concat([classification_df, feature_vector],
                                   ignore_index=True)